## **1. Introduction**

### MEGMA and AggMap
**MEGMA** is short for metagenomic **M**icrobial **E**mbedding, **G**rouping, and **M**apping **A**lgorithm (MEGMA) , which is a further step development of **AggMap** that specific for metagenomic data learning. **MEGMA** is developed to transform the tabular metagenomic data into spatially-correlated color image-like 2D-representations, named as the 2D-microbiomeprints (3D tensor data in the form of row, column and channel). 2D-microbiomeprints are multichannel feature maps (Fmaps) and are the inputs of ConvNet-based AggMapNet models. 

![MEGMA](./MEGMA.png)

**MEGMA** is released in the **aggmap** package, in this example, we will show how to employ the **aggmap** package to perform **MEGMA** on the cross-nation datasets of metagenomics.

### Metagenomic cross-nation datasets

The CRC-Nation data covers 849 gut microbial species from fecal samples of 575 subjects (CRCs or healthy controls, CTRs) compiled by Wirbel et al. (2019)3. It contains five metagenomic datasets from five separate studies in five nations (FRA: France, AUS: Australia, DEU: Germany, CHN: China, USA: America). 

In [ ]:
https://raw.githubusercontent.com/shenwanxiang/bidd-aggmap/master/docs/source/MEGMA/dataset/AUS_dfx.csv